In [ ]:
import os
import numpy as np
from PIL import Image
import pandas as pd

# Define the base directory for the slices and the output CSV file path
base_dir = '../BMP_Slices_Reduced/output-256-sub'
output_csv = './point_cloud_data.csv'

# Function to process image slices and extract non-black pixels
def process_slices(base_dir):
    data = []
    for root, dirs, files in os.walk(base_dir):
        print(f"Processing folder: {root}")
        for file in files:
            if file.endswith(".png"):
                file_path = os.path.join(root, file)
                print(f"Processing file: {file_path}")
                image = Image.open(file_path)
                image_array = np.array(image)
                non_black_pixels = np.argwhere(image_array > 0)
                for pixel in non_black_pixels:
                    data.append([os.path.basename(root), file, pixel[0], pixel[1], image_array[pixel[0], pixel[1]]])
    return data

# Process the slices and extract data
print("Starting to process image slices...")
data = process_slices(base_dir)
print("Finished processing image slices.")

# Convert the data to a DataFrame
print("Converting data to DataFrame...")
df = pd.DataFrame(data, columns=['folder', 'slice', 'x', 'y', 'intensity'])
print("Data converted to DataFrame.")

# Save the DataFrame to a CSV file
print(f"Saving DataFrame to CSV file at {output_csv}...")
df.to_csv(output_csv, index=False)
print(f"Data saved to {output_csv}")


Starting to process image slices...
Processing folder: ../BMP_Slices_Reduced/output-256-sub
Processing folder: ../BMP_Slices_Reduced/output-256-sub\scan_0004_Lesion
Processing file: ../BMP_Slices_Reduced/output-256-sub\scan_0004_Lesion\slice_0_143.png
Processing file: ../BMP_Slices_Reduced/output-256-sub\scan_0004_Lesion\slice_0_144.png
Processing file: ../BMP_Slices_Reduced/output-256-sub\scan_0004_Lesion\slice_0_145.png
Processing file: ../BMP_Slices_Reduced/output-256-sub\scan_0004_Lesion\slice_0_146.png
Processing file: ../BMP_Slices_Reduced/output-256-sub\scan_0004_Lesion\slice_0_147.png
Processing file: ../BMP_Slices_Reduced/output-256-sub\scan_0004_Lesion\slice_0_148.png
Processing file: ../BMP_Slices_Reduced/output-256-sub\scan_0004_Lesion\slice_0_149.png
Processing file: ../BMP_Slices_Reduced/output-256-sub\scan_0004_Lesion\slice_0_150.png
Processing file: ../BMP_Slices_Reduced/output-256-sub\scan_0004_Lesion\slice_0_151.png
Processing file: ../BMP_Slices_Reduced/output-256-su

In [ ]:
import os
import numpy as np
from PIL import Image
import pandas as pd
import concurrent.futures

# Define the base directory for the slices and the output CSV file path
base_dir = '../BMP_Slices_Reduced/output-256'
output_csv = './point_cloud_data.csv'

# Function to process a single image file
def process_image(file_path):
    image = Image.open(file_path)
    image_array = np.array(image)
    non_black_pixels = np.argwhere(image_array > 0)
    data = []
    for pixel in non_black_pixels:
        data.append([os.path.basename(os.path.dirname(file_path)), os.path.basename(file_path), pixel[0], pixel[1], image_array[pixel[0], pixel[1]]])
    return data

# Function to process image slices and extract non-black pixels
def process_slices(base_dir):
    data = []
    with concurrent.futures.ProcessPoolExecutor() as executor:
        futures = []
        for root, dirs, files in os.walk(base_dir):
            if files:
                print(f"Processing folder: {root}")
            for file in files:
                if file.endswith(".png"):
                    file_path = os.path.join(root, file)
                    futures.append(executor.submit(process_image, file_path))
        
        for future in concurrent.futures.as_completed(futures):
            try:
                data.extend(future.result())
            except Exception as e:
                print(f"Error processing file: {e}")
    return data

# Process the slices and extract data
print("Starting to process image slices...")
data = process_slices(base_dir)
print("Finished processing image slices.")

# Convert the data to a DataFrame
print("Converting data to DataFrame...")
df = pd.DataFrame(data, columns=['folder', 'slice', 'x', 'y', 'intensity'])
print("Data converted to DataFrame.")

# Save the DataFrame to a CSV file
print(f"Saving DataFrame to CSV file at {output_csv}...")
df.to_csv(output_csv, index=False)
print(f"Data saved to {output_csv}")


Starting to process image slices...
Processing folder: ../BMP_Slices_Reduced/output-256\scan_0004_Lesion
Processing folder: ../BMP_Slices_Reduced/output-256\scan_0004_T1
Processing folder: ../BMP_Slices_Reduced/output-256\scan_0007_Lesion
Processing folder: ../BMP_Slices_Reduced/output-256\scan_0007_T1
Processing folder: ../BMP_Slices_Reduced/output-256\scan_0016_T1
Processing folder: ../BMP_Slices_Reduced/output-256\scan_0017_Lesion
Processing folder: ../BMP_Slices_Reduced/output-256\scan_0017_T1
Processing folder: ../BMP_Slices_Reduced/output-256\scan_0019_Lesion
Processing folder: ../BMP_Slices_Reduced/output-256\scan_0019_T1
Processing folder: ../BMP_Slices_Reduced/output-256\scan_0021_Lesion
Processing folder: ../BMP_Slices_Reduced/output-256\scan_0021_T1
Processing folder: ../BMP_Slices_Reduced/output-256\scan_0024_Lesion
Processing folder: ../BMP_Slices_Reduced/output-256\scan_0024_T1
Processing folder: ../BMP_Slices_Reduced/output-256\scan_0029_Lesion
Processing folder: ../BMP_

In [ ]:
import os
import nibabel as nib
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# Define the path to the NIfTI files
data_dir = '../Aims-Tbi'

def load_nii(file_path):
    nii = nib.load(file_path)
    return nii.get_fdata()

def nii_to_point_cloud(nii_data, threshold=0):
    points = np.argwhere(nii_data > threshold)
    values = nii_data[nii_data > threshold]
    return points, values

def load_data(data_dir):
    t1_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if 'T1' in f]
    lesion_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if 'Lesion' in f]
    
    all_points = []
    all_values = []
    all_labels = []
    
    for t1_file, lesion_file in zip(t1_files, lesion_files):
        t1_data = load_nii(t1_file)
        lesion_data = load_nii(lesion_file)
        
        points, values = nii_to_point_cloud(t1_data)
        labels = lesion_data[points[:, 0], points[:, 1], points[:, 2]]
        
        print(points, values, labels)
        # all_points.append(points)
        # all_values.append(values)
        # all_labels.append(labels)
    
    # print( all_points, all_values, all_labels)

# Load the data
load_data(data_dir)


[[ 40 103 204]
 [ 40 104 202]
 [ 40 104 203]
 ...
 [215 209  49]
 [215 209  50]
 [215 210  48]] [3.88642192 3.10969901 2.3321588  ... 7.7382369  9.28341866 8.51852131] [0. 0. 0. ... 0. 0. 0.]
[[ 49  69 168]
 [ 49  72 126]
 [ 49  72 127]
 ...
 [207 185  78]
 [207 185  82]
 [207 186  77]] [14.17558002 14.17558002 14.17558002 ... 14.17558002 14.17558002
 14.17558002] [0. 0. 0. ... 0. 0. 0.]
[[  0  28   0]
 [  0  28   1]
 [  0  28   2]
 ...
 [255 227  37]
 [255 227  38]
 [255 227  39]] [  7.67820024   7.83886003   5.63396788 ... 221.2767334    8.65059948
   6.87491226] [0. 0. 0. ... 0. 0. 0.]
[[ 40  10  28]
 [ 40  10  29]
 [ 40  10  30]
 ...
 [215 244 225]
 [215 244 226]
 [215 244 227]] [ 7. 10. 11. ...  2.  5.  4.] [0. 0. 0. ... 0. 0. 0.]
[[ 41  29 196]
 [ 41  29 197]
 [ 41  30 196]
 ...
 [215 243 190]
 [215 243 191]
 [215 243 192]] [5.19759798 5.19245815 5.21915913 ... 8.20855999 8.19331551 8.17810249] [0. 0. 0. ... 0. 0. 0.]
[[ 31   0  40]
 [ 31   0  41]
 [ 31   0  42]
 ...
 [225 253 24